 ## Feature Engineering

Dưới đây là một số ý tưởng về cách tạo mới và chọn thuộc tính cho mô hình mà nhóm tác giả đã thực hiện:
>Kernel https://www.kaggle.com/kyakovlev/ieee-fe-for-local-test của Konstantin thực hiện các FE for local test và chứng minh các kỹ thuật đó. Mọi người có thể xem để tham khảo thêm
1. Tạo một feature mới và sau đó đánh giá nó bằng các tiến hành xác thực cục bộ local validation scheme  để xem liệu nó có cải thiện CV của mô hình hay không. Giữ lại các feature có lợi và loại bỏ các feature khác.
2. Lựa chọn các thuộc tính chuyển tiếp (sử dụng trên một thuộc tính hoặc nhóm các thuộc tính)
3. Loại bỏ thuộc tính đệ quy (sử dụng cho một thuộc tính hoặc nhóm các thuộc tính)
4. Xác đinh tầm quan trọng của thuộc tính bằng cách thực hiện hoán vị các giá trị thuộc tính.
5. Xác thực chéo.
6. Phân tích tương quan các thuộc tính.
7. Tính nhất quán về thời gian/
8. Tính nhất quán của khách hàng.
9. Kiểm tra sự phân phối của tập dữ liệu thử nghiệm so với tập dữ liệu đào tạo và nếu có thể xem chúng khác nhau như thế nào.

### Train & Test
>Khi thực hiện mã hóa nhãn, phải mã hóa trên cùng tập dữ liệu Train và Test bằng cách gộp chung dữ liệu lại với nhau.

In [ ]:
df = pd.concat([train[col],test[col]],axis=0)
# PERFORM FEATURE ENGINEERING HERE
train[col] = df[:len(train)]
test[col] = df[len(train):]

### NAN Processing
>Nếu ta cung cấp giá trị np.nan cho LGBM/XGB, thì tại mỗi lần Tree phân chia node, nó sẽ tách các giá trị không phải NAN và sau đó gửi tất cả các NAN đến con trái hoặc con phải tùy thuộc vào điều gì tốt nhất. Do đó, các giá trị NAN được đối xử đặc biệt ở mọi node và có thể trở dẫn đến overfit.

-->Chuyển đổi tất cả NAN thành một số âm thấp hơn tất cả các giá trị không phải NAN (chẳng hạn như - 999) thì mô hình sẽ không ưu tiên xử lý với NAN nữa và dành cho nó sự chú ý giống như những con số khác.

In [ ]:
df[col].fillna(-999, inplace=True)

### Label Encode/ Factorize/ Memory reduction
Factorizing -Mã hóa nhãn : chuyển đổi một cột kiểu string, category, object thành integers. Sau đó, ta có thể chuyển thành kiểu int8, int16 hoặc int32 tùy thuộc vào việc kiểm tra giá trị lớn nhất của cột. Từ đó giúp giảm bộ nhớ lưu trữ.

In [ ]:
df[col],_ = df[col].factorize()

if df[col].max()<128: df[col] = df[col].astype('int8')
elif df[col].max()<32768: df[col] = df[col].astype('int16')
else: df[col].astype('int32')

Ngoài ra để giảm bộ nhớ, ta sử dụng memory_reducer trên các cột. Một cách tiếp cận đơn giản và an toàn hơn là chuyển đổi tất cả các giá trị kiểu float64 thành float32 và tất cả giá trị kiểu int64 thành int32. (Tốt nhất nên tránh float16 , sử dụng int8 và int16 nếu muốn).

In [ ]:
for col in df.columns:
    if df[col].dtype=='float64': df[col] = df[col].astype('float32')
    if df[col].dtype=='int64': df[col] = df[col].astype('int32')

### Categorical Features
Với các biến phân loại, ta có thể lựa chọn cho mô hình khi biết rằng chúng vó kiểu 'categorical' hoặc có thể coi nó là kiểu 'numeric' nếu ta đã mã hóa các nhãn của nó trước đó. Dù bằng cách nào, mô hình cũng có thể trích xuất các categorical classes.

### Splitting
Một cột kiểu string/numeric có thể được tách thành hai cột mới.

>Ví dụ, một cột feature kiểu string chẳng hạn như cột 'id_30' có gí trị “ Mac OS X 10_9_5”  có thể được chia thành 2 cột mới là:
>1. "Operating System"  có giá trị "Mac OS X” 
>2. "Version" có giá trị là  “10_9_5”. 
>
>Hoặc ví dụ với cột feature là "TransactionAmt" kiểu numeric có giá trị “ 1230.45” có thể được tách thành 2 cột ( tức là 2 feature mới) :
>1. "USD" có giá trị  “ 1230” 
>2. "Cent" có giá trị  “ 45”


### Combining / Transforming / Interaction
Hai cột có kiểu string/ numeric có thể được kết hợp thành một cột mới. 

>Ví dụ cột feature "card1" và feature "card2" có thể trở thành một cột feature mới là "uid"

In [ ]:
df['uid'] = df[‘card1’].astype(str)+’_’+df[‘card2’].astype(str)

>Điều này giúp ích cho mô hình bởi vì bản thân thuộc tính 'card1' và 'card2' có thể không tương quan với target của bài toán ,do đó mô hình sẽ không phân chia các thuộc tính này. Nhưng thuộc tính 'uid = card1_card2' có thể tương quan với target và bây giờ mô hình có thể sử dụng thuộc tính này.
>
> Các cột thuộc tính kiểu số - numeric có thể được kết hợp với nhau bằng các phép cộng, trừ, nhân, v.v. 
>
>ví dụ:

In [ ]:
 df['x1_x2'] = df['x1'] * df['x2']

### Frequency Encoding
>Mã hóa tần số là một kỹ thuật mạnh mẽ cho phép mô hình "nhìn thấy" liệu các giá trị nào đó của thuộc tính là hiếm gặp hay phổ biến. 
>
>Ví dụ: nếu muốn mô hình nhân ra thuộc tính "credit_card" nào được sử dụng thường xuyên t có thể tiến hành đếm tần số xuất hiện của tất cả các giá trị cột "card1":

In [ ]:
temp = df['card1'].value_counts().to_dict()
df['card1_counts'] = df['card1'].map(temp)

### Aggregations / Group Statistics
>Cung cấp cho mô hình các thông kê giá trị theo nhóm thuộc tính cho phép mô hình xác định xem giá trị nào là phổ biến hay hiếm gặp đối với từng nhóm thuộc tính cụ thể.

In [ ]:
temp = df.groupby('card1')['TransactionAmt'].agg(['mean'])   
    .rename({'mean':'TransactionAmt_card1_mean'},axis=1)
df = pd.merge(df,temp,on='card1',how='left')

### Normalize / Standardize
>Chuẩn hóa các cột so với chính chúng. Hoặc chuẩn hóa cột này so với một cột khác.

In [ ]:
#themselfs 
df[col] = ( df[col]-df[col].mean() ) / df[col].std()

#normalize D3
df['D3_remove_time'] = df['D3'] - df['D3_week_mean']

### Outlier Removal / Relax / Smooth / PCA
>Trong cuộc thi này, tác giả muốn tìm ra điểm bất thường nên tg sử dụng kỹ thuật làm mịn một cách cẩn thận.
>
>Ý tưởng : xác định và thay thế các giá trị không phổ biến.
>
>Ví dụ: bằng cách sử dụng mã hóa tần suất cho một biến, ta có thể loại bỏ tất cả các giá trị xuất hiện dưới 0,1% bằng cách thay thế chúng bằng một giá trị mới như -9999. (khác giá trị thay cho NAN)